In [26]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [27]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [33]:
X_test.shape

(5160, 8)

In [39]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel(30, activation="relu")

In [40]:
np.random.seed(42)
tf.random.set_seed(42)

In [42]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

Train on 11610 samples, validate on 3870 samples
Epoch 1/10
11610/11610 [==============================] - 2s 155us/sample - loss: 0.5267 - output_1_loss: 0.4592 - output_2_loss: 1.1334 - val_loss: 0.6452 - val_output_1_loss: 0.4340 - val_output_2_loss: 2.5436
Epoch 2/10
11610/11610 [==============================] - 1s 71us/sample - loss: 0.5171 - output_1_loss: 0.4521 - output_2_loss: 1.1027 - val_loss: 0.5987 - val_output_1_loss: 0.4235 - val_output_2_loss: 2.1736
Epoch 3/10
11610/11610 [==============================] - 1s 73us/sample - loss: 0.5092 - output_1_loss: 0.4460 - output_2_loss: 1.0765 - val_loss: 0.5631 - val_output_1_loss: 0.4148 - val_output_2_loss: 1.8966
Epoch 4/10
11610/11610 [==============================] - 1s 75us/sample - loss: 0.5028 - output_1_loss: 0.4417 - output_2_loss: 1.0513 - val_loss: 0.5360 - val_output_1_loss: 0.4166 - val_output_2_loss: 1.6094
Epoch 5/10
11610/11610 [==============================] - 1s 71us/sample - loss: 0.4971 - output_1_loss: 0